# Зимний университет 2024. Введение в Deep learning.

Лабораторная состоит из пошагового гайда по методам глубинного обучения (Deep Learning) на Python для изучение LSTM на задаче прогнозирования стоимости акций:

1. Иморт необходимых пакетов
2. Загрузка и проверка данных
3. Нанесение данных на график
4. Предварительная обработака данных
5. Создание последовательностей и меток для обучения и теста
6. Создание и обучение модели на базе LSTM
7. Оценка эффектинвости модели
8. Прогноз с помощью модели и визуализация

Первое задание:
На шаге 7 можно изменить параметры модели для повышения ее качества. По нему будет выстроен рейтинг.

Дополнительное задание:
Изменить модель на GRU и протестировать ее (+5 баллов).

Итоговый балл считается следующим образом: рассчитывается балл за первое задание (= кол-во сдавших - местро в рейтинге + 1), к нему добавляются баллы за доп задание.

По завершению каждого задания - подзывайте преподавателя для демонстрации работы.

В этом ноутбуке изначально опущены результаты исполнения кода. Рекомендуется запускать (Shift+Enter) ячейки по мере просмотра документа

In [ ]:
!pip install --upgrade pip

# Шаг 1: Импорт библиотек и настройка конфигурации
Этот шаг включает импорт различных библиотек, необходимых для задач обработки данных, визуализации, машинного обучения и глубокого обучения.


In [ ]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, r2_score
from collections import defaultdict

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

Colour_Palette = ['#01BEFE', '#FF7D00', '#FFDD00', '#FF006D', '#ADFF02', '#8F00FF']
sns.set_palette(sns.color_palette(Colour_Palette))

tqdm.pandas()

# Шаг 2: Загрузка и проверка данных
Загрузите исторические данные о ценах акций Apple Inc. (AAPL) из Yahoo Finance. Проверьте данные с помощью df.head()и , df.info()чтобы понять их структуру и содержание.

In [ ]:
import yfinance as yf
from datetime import date

end_date = date.today().strftime("%Y-%m-%d")
start_date = '1990-01-01'

df = yf.download('AAPL', start=start_date, end=end_date)

# Inspect the data
print(df.head())
print(df.info())

# Шаг 3: Нанесение данных на график
Определите функцию для построения графика данных с использованием линейных графиков для каждого столбца в DataFrame. Это помогает визуализировать тенденции и закономерности в данных.

In [ ]:
import matplotlib.dates as mdates

def data_plot(df):
    # Plot line charts
    df_plot = df.copy()

    ncols = 2
    nrows = int(round(df_plot.shape[1] / ncols, 0))

    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, figsize=(14, 7))
    for i, ax in enumerate(fig.axes):
        sns.lineplot(data=df_plot.iloc[:, i], ax=ax)
        ax.tick_params(axis="x", rotation=30, labelsize=10, length=0)
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    fig.tight_layout()
    plt.show()

# Plot the data
data_plot(df)

# Шаг 4: Предварительная обработка данных
На этом этапе мы разделяем данные на обучающие и тестовые наборы и нормализуем значения с помощью MinMaxScaler. Эта предварительная обработка необходима для подготовки данных для моделей машинного обучения

- math.ceil: Используется для расчета количества точек обучающих данных (80% от общего числа данных)
- train_data и test_data: Разделить DataFrame на обучающий и тестовый наборы
- MinMaxScaler: Масштабирует данные до диапазона [0, 1]. Эта нормализация помогает нейронной сети сходиться быстрее

In [ ]:
import math
from sklearn.preprocessing import MinMaxScaler

# Train test split
training_data_len = math.ceil(len(df) * .8)
print(training_data_len)

# Splitting the dataset
train_data = df[:training_data_len].iloc[:, :1]
test_data = df[training_data_len:].iloc[:, :1]
print(train_data.shape, test_data.shape)

# Selecting Open Price values
dataset_train = train_data.values
# Reshaping 1D to 2D array
dataset_train = np.reshape(dataset_train, (-1, 1))
print(dataset_train.shape)

# Selecting Open Price values
dataset_test = test_data.values
# Reshaping 1D to 2D array
dataset_test = np.reshape(dataset_test, (-1, 1))
print(dataset_test.shape)

scaler = MinMaxScaler(feature_range=(0, 1))
# Scaling dataset
scaled_train = scaler.fit_transform(dataset_train)
print(scaled_train[:5])

# Normalizing values between 0 and 1
scaled_test = scaler.fit_transform(dataset_test)
print(scaled_test[:5])

# Шаг 5: Создание последовательности и меток для обучения и тестирования
Мы структурируем данные в последовательности для модели LSTM. Каждая последовательность содержит заданное количество временных шагов. Затем мы преобразуем данные в тензоры PyTorch, которые необходимы для ввода в модель PyTorch.

- sequence_length: Количество временных шагов, на которые модель оглядывается назад, чтобы сделать прогноз (можно изменять, для повышения качества модели).
- X_train и y_train: Массивы для хранения входных последовательностей и соответствующих им меток для обучения.
- X_test и y_test: Массивы для тестирования данных.
torch.tensor: Преобразует массивы numpy в тензоры PyTorch.

In [ ]:
# Create sequences and labels for training data
sequence_length = 50  # Number of time steps to look back
X_train, y_train = [], []
for i in range(len(scaled_train) - sequence_length):
    X_train.append(scaled_train[i:i + sequence_length])
    y_train.append(scaled_train[i + sequence_length])  # Predicting the value right after the sequence
X_train, y_train = np.array(X_train), np.array(y_train)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
print(X_train.shape, y_train.shape)

# Create sequences and labels for testing data
sequence_length = 30  # Number of time steps to look back
X_test, y_test = [], []
for i in range(len(scaled_test) - sequence_length):
    X_test.append(scaled_test[i:i + sequence_length])
    y_test.append(scaled_test[i + sequence_length])  # Predicting the value right after the sequence
X_test, y_test = np.array(X_test), np.array(y_test)

# Convert data to PyTorch tensors
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
print(X_test.shape, y_test.shape)

# Шаг 6: Создание и обучение модели LSTM
Определите модель LSTM для прогнозирования временных рядов. Модель включает в себя слой LSTM, за которым следует полносвязный слой. Обучите модель на обучающих данных и оцените её на тестовых данных.

- LSTMModel: класс нейронной сети PyTorch со слоем LSTM и линейным слоем.
Конфигурация устройства: проверьте, доступен ли графический процессор, и используйте его, если возможно.
- Гиперпараметры: такие настройки, как размер входных данных, размер скрытых данных, количество слоёв, коэффициент отбрасывания, размер пакета, скорость обучения и количество эпох (можно изменять).
- DataLoader: утилита для пакетной обработки и перемешивания набора данных.
Цикл обучения: цикл по набору данных в течение заданного количества эпох, выполняющий прямой и обратный проходы и обновляющий веса модели.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

input_size = 1
num_layers = 3  # Число слоев в модели
hidden_size = 128  # Размерность скрытого состояния
output_size = 1
dropout = 0.2  # Добавочный dropout для регуляризации

model = LSTMModel(input_size, hidden_size, num_layers, dropout).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Learning rate

batch_size = 32  # Adjusted batch size
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

num_epochs = 20  # Increased number of epochs
train_hist = []
test_hist = []

for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        predictions = model(batch_X)
        loss = loss_fn(predictions, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    train_hist.append(average_loss)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_loss:.4f}')

График обучения

In [ ]:
x = np.linspace(1,num_epochs,num_epochs)
plt.plot(x,train_hist,scalex=True, label="Training loss")
plt.legend()
plt.show()

# Шаг 7: Оцените эффективность модели с помощью таких показателей, как среднеквадратическая ошибка и коэффициент R².



In [ ]:
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Evaluate the model and calculate RMSE and R² score
model.eval()
with torch.no_grad():
    test_predictions = []
    for batch_X_test in X_test:
        batch_X_test = batch_X_test.to(device).unsqueeze(0)  # Add batch dimension
        test_predictions.append(model(batch_X_test).cpu().numpy().flatten()[0])

test_predictions = np.array(test_predictions)

# Calculate RMSE and R² score
rmse = np.sqrt(mean_squared_error(y_test.cpu().numpy(), test_predictions))
r2 = r2_score(y_test.cpu().numpy(), test_predictions)

print(f'RMSE: {rmse:.4f}')
print(f'R² Score: {r2:.4f}')

# Шаг 8: Спрогнозируйте будущие значения и оцените модель
Используйте обученную модель для прогнозирования будущих значений.

- Прогнозирование: генерирование будущих значений с использованием обученной модели.
- Построение графиков: визуализируйте фактические и прогнозируемые значения.

Парматеры, которые можно поменять:
- steps - сдвиг от конца последовательности (чем больше, тем больше пересечение с известными данными)
- num_forecast_steps - число предсказываемых измерений (если больше, чем steps можно заглянуть в будущее)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import mean_squared_error, r2_score

steps = 30

num_forecast_steps = 60
sequence_to_plot = X_test.squeeze().cpu().numpy()
historical_data = sequence_to_plot[-steps]

forecasted_values = []
with torch.no_grad():
    for _ in range(num_forecast_steps):
        historical_data_tensor = torch.as_tensor(historical_data).view(1, -1, 1).float().to(device)
        predicted_value = model(historical_data_tensor).cpu().numpy()[0, 0]
        forecasted_values.append(predicted_value)
        historical_data = np.roll(historical_data, shift=-1)
        historical_data[-1] = predicted_value

last_date = test_data.index[-steps]
future_dates = pd.date_range(start=last_date + pd.DateOffset(1), periods=num_forecast_steps)

forecasted_values_scaled = scaler.inverse_transform(np.array(forecasted_values).reshape(-1, 1)).flatten()
combined_values = np.concatenate([test_data[-steps:-steps+1].values.flatten(), forecasted_values_scaled])

plt.rcParams['figure.figsize'] = [14, 4]
plt.plot(test_data.index[-100:], test_data[-100:], label="test_data", color="b")
plt.plot(test_data.index[-steps:-steps+1].append(future_dates), combined_values, label='forecasted values', color='red')
plt.plot(test_data.index[-steps:], test_data[-steps:], label='actual values', color='green')

plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.title('Time Series Forecasting')
plt.grid(True)
plt.show()
